# Clasificador de reseñas

- **Joan Pedro Bruxola**
- **Julio García Bustos**
- **Marcos Rosique Martínez**
- **Mario Soto Ramos**

In [17]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
from langdetect import detect, LangDetectException

## Extracción de URLs

### URLs de las películas

In [ ]:
# Parámetros para la URL (puedes cambiar estos valores dinámicamente)
params = {
    "releaseYearMin": 2015,  # Año mínimo
    "releaseYearMax": 2020,  # Año máximo
    "page": 1                # Número de página
}

n = 24  # Número de películas que queremos obtener

# URL base
url = "https://www.metacritic.com/browse/movie/"

# Realizamos la solicitud GET con los parámetros
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}
response = requests.get(url, headers=headers, params=params)

# Comprobamos si la solicitud fue exitosa
if response.status_code == 200:
    soup = BeautifulSoup(response.content, "html.parser")
    
    # Encontramos todos los enlaces a las películas
    movie_links_positive = soup.find_all("a", class_="c-finderProductCard_container g-color-gray80 u-grid")
    
    # Lista para almacenar los enlaces
    links_positive = []

    # Extraemos los enlaces de las primeras 24 películas y los guardamos en la lista
    for idx, link in enumerate(movie_links_positive[:n]):  # Solo las primeras 24
        href = link.get("href")
        if href:
            full_url = f"https://www.metacritic.com{href}"
            links_positive.append(full_url)  # Guardamos el enlace en la lista
            print(f"Película {idx+1}: {full_url}")
    
    # Opcional: Puedes ver todos los enlaces almacenados
    print("\nLista de enlaces almacenados:")
    print(links_positive)

else:
    print("Error al acceder a la página.")

Película 1: https://www.metacritic.com/movie/moonlight-2016/
Película 2: https://www.metacritic.com/movie/parasite/
Película 3: https://www.metacritic.com/movie/roma/
Película 4: https://www.metacritic.com/movie/manchester-by-the-sea/
Película 5: https://www.metacritic.com/movie/small-axe-lovers-rock/
Película 6: https://www.metacritic.com/movie/portrait-of-a-lady-on-fire/
Película 7: https://www.metacritic.com/movie/i-am-not-your-negro/
Película 8: https://www.metacritic.com/movie/collective/
Película 9: https://www.metacritic.com/movie/carol/
Película 10: https://www.metacritic.com/movie/amazing-grace-1972/
Película 11: https://www.metacritic.com/movie/dunkirk/
Película 12: https://www.metacritic.com/movie/45-years/
Película 13: https://www.metacritic.com/movie/the-irishman/
Película 14: https://www.metacritic.com/movie/inside-out-2015/
Película 15: https://www.metacritic.com/movie/faces-places/
Película 16: https://www.metacritic.com/movie/call-me-by-your-name/
Película 17: https://

In [ ]:
# Parámetros para la URL (puedes cambiar estos valores dinámicamente)
params = {
    "releaseYearMin": 2015,  # Año mínimo
    "releaseYearMax": 2020,  # Año máximo
    "page": 149                # Número de página
}

# URL base
url = "https://www.metacritic.com/browse/movie/"

# Realizamos la solicitud GET con los parámetros
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}
response = requests.get(url, headers=headers, params=params)

# Comprobamos si la solicitud fue exitosa
if response.status_code == 200:
    soup = BeautifulSoup(response.content, "html.parser")
    
    # Encontramos todos los enlaces a las películas
    movie_links_worst = soup.find_all("a", class_="c-finderProductCard_container g-color-gray80 u-grid")
    
    # Lista para almacenar los enlaces
    links_worst = []

    # Extraemos los enlaces de las primeras 24 películas y los guardamos en la lista
    for idx, link in enumerate(movie_links_worst[:n]):  # Solo las primeras 24
        href = link.get("href")
        if href:
            full_url = f"https://www.metacritic.com{href}"
            links_worst.append(full_url)  # Guardamos el enlace en la lista
            print(f"Película {idx+1}: {full_url}")
    
    # Opcional: Puedes ver todos los enlaces almacenados
    print("\nLista de enlaces almacenados:")
    print(links_worst)

else:
    print("Error al acceder a la página.")

Película 1: https://www.metacritic.com/movie/search-party/
Película 2: https://www.metacritic.com/movie/max-steel/
Película 3: https://www.metacritic.com/movie/enter-the-dangerous-mind/
Película 4: https://www.metacritic.com/movie/home-sweet-hell/
Película 5: https://www.metacritic.com/movie/martyrs/
Película 6: https://www.metacritic.com/movie/the-tax-collector/
Película 7: https://www.metacritic.com/movie/gods-not-dead-2/
Película 8: https://www.metacritic.com/movie/blumhouses-fantasy-island/
Película 9: https://www.metacritic.com/movie/norm-of-the-north/
Película 10: https://www.metacritic.com/movie/a-family-man/
Película 11: https://www.metacritic.com/movie/geostorm/
Película 12: https://www.metacritic.com/movie/dirty-grandpa/
Película 13: https://www.metacritic.com/movie/211/
Película 14: https://www.metacritic.com/movie/the-clapper/
Película 15: https://www.metacritic.com/movie/just-getting-started/
Película 16: https://www.metacritic.com/movie/life-itself-2018/
Película 17: http

In [ ]:
# Parámetros para la URL (puedes cambiar estos valores dinámicamente)
params = {
    "releaseYearMin": 2015,  # Año mínimo
    "releaseYearMax": 2020,  # Año máximo
    "page": 90                # Número de página
}

# URL base
url = "https://www.metacritic.com/browse/movie/"

# Realizamos la solicitud GET con los parámetros
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}
response = requests.get(url, headers=headers, params=params)

# Comprobamos si la solicitud fue exitosa
if response.status_code == 200:
    soup = BeautifulSoup(response.content, "html.parser")
    
    # Encontramos todos los enlaces a las películas
    movie_links_neutral = soup.find_all("a", class_="c-finderProductCard_container g-color-gray80 u-grid")
    
    # Lista para almacenar los enlaces
    links_neutral = []

    # Extraemos los enlaces de las primeras 24 películas y los guardamos en la lista
    for idx, link in enumerate(movie_links_neutral[:n]):  # Solo las primeras 24
        href = link.get("href")
        if href:
            full_url = f"https://www.metacritic.com{href}"
            links_neutral.append(full_url)  # Guardamos el enlace en la lista
            print(f"Película {idx+1}: {full_url}")
    
    # Opcional: Puedes ver todos los enlaces almacenados
    print("\nLista de enlaces almacenados:")
    print(links_neutral)

else:
    print("Error al acceder a la página.")

Película 1: https://www.metacritic.com/movie/i-smile-back/
Película 2: https://www.metacritic.com/movie/between-two-ferns-the-movie/
Película 3: https://www.metacritic.com/movie/all-is-true/
Película 4: https://www.metacritic.com/movie/champs/
Película 5: https://www.metacritic.com/movie/amnesia-2015/
Película 6: https://www.metacritic.com/movie/crooked-house/
Película 7: https://www.metacritic.com/movie/judy-punch/
Película 8: https://www.metacritic.com/movie/ottolenghi-and-the-cakes-of-versailles/
Película 9: https://www.metacritic.com/movie/doctor-sleep/
Película 10: https://www.metacritic.com/movie/the-boy-downstairs/
Película 11: https://www.metacritic.com/movie/the-rachel-divide/
Película 12: https://www.metacritic.com/movie/extraordinary-tales/
Película 13: https://www.metacritic.com/movie/sing/
Película 14: https://www.metacritic.com/movie/lizzie/
Película 15: https://www.metacritic.com/movie/the-lovebirds/
Película 16: https://www.metacritic.com/movie/on-the-basis-of-sex/
Pelí

### URLs de las reseñas 

In [ ]:
# Definir los filtros y las listas
filters = ['Positive%20Reviews', 'Mixed%20Reviews', 'Negative%20Reviews']
link_lists = [links_positive, links_neutral, links_worst]

# Lista para almacenar los enlaces
review_links = []

# Iterar sobre los filtros y las listas de enlaces
for filter_type in filters:
    for link_list in link_lists:
        review_links += [f"{url}user-reviews/?filter={filter_type}" for url in link_list]

# Ahora review_links contiene todas las URLs concatenadas
print(review_links)
print(f"Tamaño de review_links: {len(review_links)}")

['https://www.metacritic.com/movie/moonlight-2016/user-reviews/?filter=Positive%20Reviews', 'https://www.metacritic.com/movie/parasite/user-reviews/?filter=Positive%20Reviews', 'https://www.metacritic.com/movie/roma/user-reviews/?filter=Positive%20Reviews', 'https://www.metacritic.com/movie/manchester-by-the-sea/user-reviews/?filter=Positive%20Reviews', 'https://www.metacritic.com/movie/small-axe-lovers-rock/user-reviews/?filter=Positive%20Reviews', 'https://www.metacritic.com/movie/portrait-of-a-lady-on-fire/user-reviews/?filter=Positive%20Reviews', 'https://www.metacritic.com/movie/i-am-not-your-negro/user-reviews/?filter=Positive%20Reviews', 'https://www.metacritic.com/movie/collective/user-reviews/?filter=Positive%20Reviews', 'https://www.metacritic.com/movie/carol/user-reviews/?filter=Positive%20Reviews', 'https://www.metacritic.com/movie/amazing-grace-1972/user-reviews/?filter=Positive%20Reviews', 'https://www.metacritic.com/movie/dunkirk/user-reviews/?filter=Positive%20Reviews',

## Extracción y almacenamiento de reseñas

In [ ]:
# Número de reseñas por película
n_res = 20

In [ ]:
def get_user_reviews_selenium(movie_url, max_pages=1):  # Cambié max_pages a 1 para limitar la cantidad de páginas
    chrome_options = Options()
    #chrome_options.add_argument("--headless")  # Ejecutar sin abrir ventana
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("user-agent=Mozilla/5.0")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

    all_reviews = []

    for page in range(max_pages):  # Solo una página por URL
        url = f"{movie_url}&page={page + 1}"  # Se añade la página actual a la URL
        driver.get(url)
        time.sleep(2)  # Espera a que cargue JavaScript

        soup = BeautifulSoup(driver.page_source, "html.parser")

        review_containers = soup.find_all("div", attrs={"data-testid": "product-review"})

        for container in review_containers:
            score_tag = container.find("div", class_="c-siteReviewScore")
            score = score_tag.get_text(strip=True) if score_tag else "No score"

            user_tag = container.find("a", class_="c-siteReviewHeader_username")
            username = user_tag.get_text(strip=True) if user_tag else "No username"

            date_tag = container.find("div", class_="c-siteReviewHeader_reviewDate")
            date = date_tag.get_text(strip=True) if date_tag else "No date"

            quote_tag = container.find("div", class_="c-siteReview_quote")
            review_text = quote_tag.get_text(strip=True) if quote_tag else "No review text"

            all_reviews.append({
                "movie_url": movie_url,  # Añadimos la URL de la película
                "username": username,
                "date": date,
                "score": score,
                "review": review_text
            })

        if len(all_reviews) >= n_res:  # Si ya tenemos 20 reseñas, salimos del bucle
            break

    driver.quit()
    return all_reviews


# Lista para almacenar todas las reseñas
all_reviews_data = []

# Extraer 10 reseñas para cada enlace en review_links
for movie_url in review_links:
    print(f"Extrayendo reseñas de: {movie_url}")
    reviews = get_user_reviews_selenium(movie_url)

    # Almacenar las reseñas en la lista
    for review in reviews[:n_res]:  # Almacenamos solo las primeras 10 reseñas
        all_reviews_data.append(review)

# Convertir la lista de reseñas a un DataFrame de pandas
df_reviews = pd.DataFrame(all_reviews_data)

# Mostrar las primeras filas del DataFrame
print(df_reviews.head())

Extrayendo reseñas de: https://www.metacritic.com/movie/moonlight-2016/user-reviews/?filter=Positive%20Reviews
Extrayendo reseñas de: https://www.metacritic.com/movie/parasite/user-reviews/?filter=Positive%20Reviews
Extrayendo reseñas de: https://www.metacritic.com/movie/roma/user-reviews/?filter=Positive%20Reviews
Extrayendo reseñas de: https://www.metacritic.com/movie/manchester-by-the-sea/user-reviews/?filter=Positive%20Reviews
Extrayendo reseñas de: https://www.metacritic.com/movie/small-axe-lovers-rock/user-reviews/?filter=Positive%20Reviews
Extrayendo reseñas de: https://www.metacritic.com/movie/portrait-of-a-lady-on-fire/user-reviews/?filter=Positive%20Reviews
Extrayendo reseñas de: https://www.metacritic.com/movie/i-am-not-your-negro/user-reviews/?filter=Positive%20Reviews
Extrayendo reseñas de: https://www.metacritic.com/movie/collective/user-reviews/?filter=Positive%20Reviews
Extrayendo reseñas de: https://www.metacritic.com/movie/carol/user-reviews/?filter=Positive%20Reviews

In [ ]:
# Guardar el DataFrame en un archivo CSV
df_reviews.to_csv("reviews_finales.csv", index=True)

## Preprocesamiento

In [29]:
# Leer el archivo CSV y establecer la columna 'Unnamed: 0' como índice
df = pd.read_csv("reviews_finales.csv")

In [30]:
df.head()

,movie_url,username,date,score,review
0,https://www.metacritic.com/movie/moonlight-201...,Randomsquirrel,"Jan 15, 2025",10,It was an absolutely brilliant film. Character...
1,https://www.metacritic.com/movie/moonlight-201...,isabellacosta8,"Oct 10, 2024",10,[SPOILER ALERT: This review contains spoilers.]
2,https://www.metacritic.com/movie/moonlight-201...,Gabrielx593,"Oct 9, 2024",10,[SPOILER ALERT: This review contains spoilers.]
3,https://www.metacritic.com/movie/moonlight-201...,NWR1,"Mar 31, 2024",9,I haven’t felt this much emotion in a long tim...
4,https://www.metacritic.com/movie/moonlight-201...,Alexmarshall,"Oct 16, 2023",10,Reallyamazing


In [31]:
df.shape

(1814, 5)

In [32]:
# Aplicar la extracción y transformación del nombre de la película a la columna "movie_url"
df['movie_url'] = df['movie_url'].apply(lambda x: re.search(r'/movie/([^/]+)', x).group(1).replace('-', ' ').title())

# Renombrar la columna "movie_url" a "movie"
df = df.rename(columns={"movie_url": "movie"})

df.head(5)

,movie,username,date,score,review
0,Moonlight 2016,Randomsquirrel,"Jan 15, 2025",10,It was an absolutely brilliant film. Character...
1,Moonlight 2016,isabellacosta8,"Oct 10, 2024",10,[SPOILER ALERT: This review contains spoilers.]
2,Moonlight 2016,Gabrielx593,"Oct 9, 2024",10,[SPOILER ALERT: This review contains spoilers.]
3,Moonlight 2016,NWR1,"Mar 31, 2024",9,I haven’t felt this much emotion in a long tim...
4,Moonlight 2016,Alexmarshall,"Oct 16, 2023",10,Reallyamazing


In [ ]:
# Función para manejar excepciones y textos vacíos
def safe_detect(text):
    try:
        if text.strip():  # Asegurarse de que el texto no esté vacío
            return detect(text)
        else:
            return None  # Si el texto está vacío, devuelve None
    except LangDetectException:
        return None  # Si no puede detectar el idioma, devuelve None

# Eliminar las filas donde la columna 'review' contiene "SPOILER ALERT" y crear la columna 'sentiment' basada en la columna 'score'
df = (df[~df['review'].str.contains("SPOILER ALERT", case=True, na=False)]
              .assign(language = df['review'].apply(safe_detect)))

# Filtrar el DataFrame para quedarse solo con las filas donde el idioma es 'en'
df = df[df['language'] == 'en']

df = df.drop('language', axis = 1)

df.head(5)

,movie,username,date,score,review
0,Moonlight 2016,Randomsquirrel,"Jan 15, 2025",10,It was an absolutely brilliant film. Character...
3,Moonlight 2016,NWR1,"Mar 31, 2024",9,I haven’t felt this much emotion in a long tim...
5,Moonlight 2016,Youngoth,"Oct 16, 2023",10,"This movie is masterpiece, so well done. Deser..."
6,Moonlight 2016,Dominik08,"Aug 15, 2023",8,"A refined production, with no obvious weakness..."
7,Moonlight 2016,AgentLvi,"Jun 27, 2023",7,"Decent movie. The story is so --so, scoring is..."


In [34]:
df['sentiment'] = df['score'].apply(lambda x: 'NEG' if x <= 3 else ('NEU' if 4 <= x <= 6 else 'POS'))

df.head(5)

,movie,username,date,score,review,sentiment
0,Moonlight 2016,Randomsquirrel,"Jan 15, 2025",10,It was an absolutely brilliant film. Character...,POS
3,Moonlight 2016,NWR1,"Mar 31, 2024",9,I haven’t felt this much emotion in a long tim...,POS
5,Moonlight 2016,Youngoth,"Oct 16, 2023",10,"This movie is masterpiece, so well done. Deser...",POS
6,Moonlight 2016,Dominik08,"Aug 15, 2023",8,"A refined production, with no obvious weakness...",POS
7,Moonlight 2016,AgentLvi,"Jun 27, 2023",7,"Decent movie. The story is so --so, scoring is...",POS


In [35]:
df.shape

(1545, 6)

Instalación del modelo de spaCy

Para instalar el modelo `en_core_web_md` de **spaCy**, ejecuta el siguiente comando en tu **Anaconda Prompt** o **Terminal**:

```bash
python -m spacy download en_core_web_md


In [39]:
import spacy
import string
nlp = spacy.load('en_core_web_md')

In [40]:
from spacy.lang.en.stop_words import STOP_WORDS
print(STOP_WORDS)

{'whom', 'either', 'almost', 'besides', 'show', 'she', 'themselves', 'keep', 'least', 'may', 'yet', 'their', 'yourselves', 'indeed', 'is', 'enough', 'or', 'ourselves', 'mine', 'well', 'them', 'under', 'were', 'out', 'all', 'more', 'will', 'then', 'by', 'across', "n't", 'ours', 'otherwise', 'most', 'because', 'two', 'although', 'noone', 'this', 'somewhere', 'whereupon', 'everything', 'yours', 'that', 'itself', 'some', 'down', 'doing', 'somehow', 'become', 'anywhere', 'unless', 'he', 'us', 'however', 'beyond', 'neither', 'thence', 'above', 'throughout', 'me', 'go', 'since', 'own', 'though', 'thru', 'meanwhile', 'in', 'must', '’m', 'latterly', 'every', 'while', 'to', 'first', 'than', 'per', 'with', 'hence', 'next', 'whence', '’d', 'various', 'do', 'formerly', 'i', 'both', 'becomes', 'serious', 'about', 'sometimes', 'a', 'nothing', 'at', "'re", 'beforehand', 'nowhere', 'you', 'thereupon', 'after', 'until', 'everyone', 'her', 'never', 'over', 'just', 'if', 'an', 'beside', 'top', 'those', 'l

In [41]:
# Definir el patrón para los caracteres especiales
pat = '[{}]'.format(re.escape(string.punctuation))

# Definir la función para normalizar, lematizar y eliminar puntuación
def normalize_document_remove_punct(doc):

    doc = re.sub(r'\d+', '', doc)
    # Separamos en tokens
    tokens = nlp(doc)
    
    # Quitamos puntuación, espacios y stopwords, y aplicamos lematización
    filtered_tokens = [re.sub(pat, ' ', t.lemma_.lower())  # Usamos t.lemma_ para lematizar
                       for t in tokens
                       if not t.is_stop        # Eliminar stopwords
                       and not t.is_punct      # Eliminar puntuación
                       and not t.is_space]      # Eliminar espacios]     
    
    # Juntamos de nuevo en una cadena
    doc = ' '.join(filtered_tokens)
    
    return doc

# Aplicar la función a cada elemento de la columna 'contenido' tanto en treino como en test
df['review'] = df['review'].apply(normalize_document_remove_punct)

df.head(5)

,movie,username,date,score,review,sentiment
0,Moonlight 2016,Randomsquirrel,"Jan 15, 2025",10,absolutely brilliant film character deep plot ...,POS
3,Moonlight 2016,NWR1,"Mar 31, 2024",9,feel emotion long time chiron manifest emotion...,POS
5,Moonlight 2016,Youngoth,"Oct 16, 2023",10,movie masterpiece deserve oscar,POS
6,Moonlight 2016,Dominik08,"Aug 15, 2023",8,refined production obvious weakness base ratio...,POS
7,Moonlight 2016,AgentLvi,"Jun 27, 2023",7,decent movie story so scoring great visual d...,POS
